# Hypothesis to test:
### Removing objective sentences from reviews helps predict star rating from reviews

In [1]:
import numpy as np
import pandas as pd
import pickle
import gzip
import math
import random
from IPython.display import Markdown, display
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor, \
GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, \
classification_report, make_scorer
import statsmodels.api as sm

In [2]:
import sys
sys.path.append('..')

In [3]:
# From this project
from utils import rmse, rmse_train_cv, classifier_report, confusion_rmse
from NLP import WordBag, AboutMovie
from subjective_filter import SubjectiveFilter

# Avoid restarting Kernel
%load_ext autoreload
%autoreload 2

pd.set_option('display.max_colwidth', -1)

# %autosave 50

## Configuration

In [4]:
data_path = '../../../datasets/'


In [5]:
pickle_in = open(data_path
                + 'movie_reviews_144000Pos_Neg_Samples.pkl'
                , "rb")
test_dic = pickle.load(pickle_in)

## Remove objective sentences for case B

In [6]:
OBJ_THRESHOLD = 0.8

In [36]:
%reload_ext autoreload
obj_path = '../obj_subj_dev/'
fit_obj_tf = obj_path + 'fit_tfidf_vectorizer_for_obj_subj_sentences_classification.pkl'
fit_obj_model = obj_path + 'GBC_300_0.5_5_0.88cv.pkl'
subj_filter = SubjectiveFilter(fit_obj_tf, fit_obj_model)

In [37]:
# for tt in ['train','test']:
#     for pn in ['positive','negative']:
#         print(tt,pn,test_dic[tt][np].shape)

for tt in test_dic.values():
    for df in tt.values():
        display(df.shape)

(40164, 5)

(41953, 5)

(10071, 5)

(10431, 5)

In [15]:
test_dic['train']['positive'].iloc[40162:40200,:]

,reviewerID,asin,reviewText,overall,words
1897726,A3CVEWCVAKE93O,B00020BVYW,"It was a good movie showing the harshness of Alaska, and the human spirit to keep going. I liked it and would recommend it.",4.0,"[wa, good, movie, showing, harshness, alaska, and, human, spirit, keep, going, liked, and, would, recommend]"
1897729,A12KWSWHLN5YE1,B00020BVYW,"We really enjoyed this movie..scary survival scenario under any circumstances, gripping right to the end. Just the right mix of fear and feel good for is.",4.0,"[really, enjoyed, movie..scary, survival, scenario, circumstance, gripping, right, end, right, mix, fear, and, feel, good]"


In [ ]:
# from IPython.display import Markdown, display

# for tt in test_dic.values():
#     for df in tt.values():
#         display(df.head(3))

In [ ]:
# type(test_dic['train']['positive']['reviewText'])

In [ ]:
# test = test_dic['train']['positive'].drop(['sentence','words'],axis=1).iloc[:2,:]  # 
# test

In [ ]:
# res = subj_filter.transform(
#             test,
#             'reviewText', 
#             threshold=OBJ_THRESHOLD,
#             debug_level=2)

In [ ]:
# tp = subj_filter.transform(
#             test_dic['train']['positive'].drop(['sentence','words'],axis=1,errors='ignore'),
#             'reviewText', 
#             threshold=OBJ_THRESHOLD,
#             debug_level=2)

In [ ]:
pickle_out = open(data_path + 'tp_subj_movie_reviews_0.8.pkl', "wb")
pickle.dump(movie_reviews, pickle_out)

In [19]:
df.iloc[:5,:]

,reviewerID,asin,reviewText,overall
861912,A3KEABH9BZ1MPB,6303625800,The adults found it very boring and the children ages 8 and 10 said it was just ok,1.0
861931,A1EBEAG4PJOIB2,6303625800,"Love this film, especially at Thanksgiving. But as long as it's only available in the phony and insulting colorized version I won't touch it. This is a piece of art. Shame on those who tamper with this and other films. It's like someone drawing a mustache on the Mona Lisa.",1.0
862419,A3MV1KKHX51FYT,6303626475,"This is the film version of the J.B. Priestly horror novel. It was a dark and stormy night, a couple is driving on a muddy country road through the Welsh mountains. Their touring car does not have side curtains. They are lost, and pull into a house with lights. The Femm household has some strange characters. This house has its own electricity. There is conflict among the household, like when prayers are offered before a meal. Then a knock on the door brings another couple to this house; they are also lost. The guests tell about themselves. [Filming in the rain must have been a technical breakthrough that year.]When they go to fetch a lamp they hear a strange sound. Is there someone in a locked room? Does the butler have a drinking problem? Will something terrible happen? If a madman is locked up what will happen if he is released? Will the innocent survive and the guilty die?This is boring and almost unwatchable film, a waste of time. It does show the performances of some actors who became more famous years later. Some of the scenes reflect the hidden quirks of the actors.",1.0
862466,AM55WQ075SIK7,6303626475,the movie called The Old Dark House should be called The Old Dark Horrible Movie. Terrible!!!!! I'm sorry I listened to the other reviews. I gave it to Goodwill.,1.0
862856,A9XKE4OE48BNK,6303631940,"This is actually neck-in-neck with Colin Baker's horrible regeneration story 'The Twin Dilemma'. This mess isn't really Sylvester McCoy's fault. The script is wretched, the plot is nonsensical (even by Who standards), and the Rani actually seems to be looking at the TV audience at one point. Ugh. It, like all of McCoy's first season, also commits the cardinal sin of being both excruciating and boring. Don't give up on McCoy though. Check out some of his stories from later in his run like Remembrance of the Daleks and Curse of Fenric.",1.0


In [24]:
res['test']['negative']

,reviewerID,asin,reviewText,overall
0,A1EBEAG4PJOIB2,6303625800,"Love this film, especially at Thanksgiving. But as long as it's only available in the phony and insulting colorized version I won't touch it. This is a piece of art. Shame on those who tamper with this and other films. It's like someone drawing a mustache on the Mona Lisa. Love this film, especially at Thanksgiving. But as long as it's only available in the phony and insulting colorized version I won't touch it. This is a piece of art. Shame on those who tamper with this and other films. It's like someone drawing a mustache on the Mona Lisa.",1.0
1,A3KEABH9BZ1MPB,6303625800,The adults found it very boring and the children ages 8 and 10 said it was just ok The adults found it very boring and the children ages 8 and 10 said it was just ok,1.0
2,A3MV1KKHX51FYT,6303626475,"This is the film version of the J.B. Priestly horror novel. The Femm household has some strange characters. This house has its own electricity. There is conflict among the household, like when prayers are offered before a meal. The guests tell about themselves. [Filming in the rain must have been a technical breakthrough that year. Is there someone in a locked room? Does the butler have a drinking problem? Will something terrible happen? If a madman is locked up what will happen if he is released? Will the innocent survive and the guilty die?This is boring and almost unwatchable film, a waste of time. It does show the performances of some actors who became more famous years later. Some of the scenes reflect the hidden quirks of the actors.",1.0
3,AM55WQ075SIK7,6303626475,the movie called The Old Dark House should be called The Old Dark Horrible Movie. Terrible!!!!! I'm sorry I listened to the other reviews. I gave it to Goodwill. the movie called The Old Dark House should be called The Old Dark Horrible Movie. Terrible!!!!! I'm sorry I listened to the other reviews. I gave it to Goodwill.,1.0
4,A9XKE4OE48BNK,6303631940,"This is actually neck-in-neck with Colin Baker's horrible regeneration story 'The Twin Dilemma'. This mess isn't really Sylvester McCoy's fault. The script is wretched, the plot is nonsensical (even by Who standards), and the Rani actually seems to be looking at the TV audience at one point. Ugh. It, like all of McCoy's first season, also commits the cardinal sin of being both excruciating and boring. Don't give up on McCoy though. Check out some of his stories from later in his run like Remembrance of the Daleks and Curse of Fenric. This is actually neck-in-neck with Colin Baker's horrible regeneration story 'The Twin Dilemma'. This mess isn't really Sylvester McCoy's fault. The script is wretched, the plot is nonsensical (even by Who standards), and the Rani actually seems to be looking at the TV audience at one point. Ugh. It, like all of McCoy's first season, also commits the cardinal sin of being both excruciating and boring. Don't give up on McCoy though. Check out some of his stories from later in his run like Remembrance of the Daleks and Curse of Fenric.",1.0


In [38]:
CHUNK_SZ = 5000
res = {'train':{},'test':{}}

for ttname, tt in test_dic.items():
    for pn, df in tt.items():
        df = df.drop(['sentence','words'],axis=1,errors='ignore')
        df_list = []
        start = 0
        while start < df.shape[0]:
            end = start + CHUNK_SZ
            df_list.append(
                subj_filter.transform(
                    df.iloc[start:end,:],
                    'reviewText', 
                    threshold=1 - OBJ_THRESHOLD,
                    debug_level=0,
                    switch=True) \
                .rename(columns={"sentence": "reviewText"}))
            start = end

        res[ttname][pn] = df_list.pop()
        while len(df_list) > 0:
            res[ttname][pn] = pd.merge(df_list.pop(), res[ttname][pn], how='outer')
#         res[ttname][pn] = reduce(lambda x, y: pd.merge(x, y, on = ['reviewerID','asin']), dfList)

../subjective_filter.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['sentence'] = df['reviewText'].map(


#### => Removed 17355 (44%) objective sentences

#### => Removed 648 (13%) reviews with no emotional content

../subjective_filter.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['sentence'] = df['reviewText'].map(


#### => Removed 16622 (42%) objective sentences

...

#### => Removed 19109 (47%) objective sentences

#### => Removed 642 (13%) reviews with no emotional content

../subjective_filter.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['sentence'] = df['reviewText'].map(


#### => Removed 1471 (45%) objective sentences

#### => Removed 47 (11%) reviews with no emotional content

In [39]:
total = 0
old_total = 0
for tt in ['train', 'test']:
    for pn in ['positive', 'negative']:
        print('\n {} {}'.format(tt,pn))
        tot = test_dic[tt][pn].shape[0]
        new = res[tt][pn].shape[0]
        dropped = tot - new
        print('Dropped: {0} ({1:.1%})'.format(dropped, dropped/tot))
        total += new
        old_total += tot
print('\nOld total:', old_total)
print('New total:', total)
print('Removed a total of:', old_total - total)


 train positive
Dropped: 5556 (13.8%)

 train negative
Dropped: 5560 (13.3%)

 test positive
Dropped: 1484 (14.7%)

 test negative
Dropped: 1371 (13.1%)

Old total: 102619
New total: 88648
Removed a total of: 13971


In [40]:
pickle_out = open(data_path
                    + 'obj_movie_reviews_0.8.pkl'
                    , "wb")
pickle.dump(res, pickle_out)
pickle_out.close()